In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [41]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [42]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

print(encode('hello'))
print(decode(encode('hello')))

[61, 58, 65, 65, 68]
hello


In [43]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])
print(data.shape)

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])
torch.Size([232311])


In [44]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ', x)
print('targets: ', y)

inputs:  tensor([[54, 65, 65,  1, 54, 67, 57,  1],
        [61, 62, 66, 72, 58, 65, 59,  0],
        [ 0, 55, 58, 57, 72,  1, 54, 73],
        [67,  0, 54, 67, 57,  1, 72, 69]], device='cuda:0')
targets:  tensor([[65, 65,  1, 54, 67, 57,  1, 73],
        [62, 66, 72, 58, 65, 59,  0, 59],
        [55, 58, 57, 72,  1, 54, 73,  1],
        [ 0, 54, 67, 57,  1, 72, 69, 58]], device='cuda:0')


In [45]:
'''
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('When Input is ', context, 'target is ', target) 

Output ->
When Input is  tensor([80]) target is  tensor(1)
When Input is  tensor([80,  1]) target is  tensor(1)
When Input is  tensor([80,  1,  1]) target is  tensor(28)
When Input is  tensor([80,  1,  1, 28]) target is  tensor(39)
When Input is  tensor([80,  1,  1, 28, 39]) target is  tensor(42)
When Input is  tensor([80,  1,  1, 28, 39, 42]) target is  tensor(39)
When Input is  tensor([80,  1,  1, 28, 39, 42, 39]) target is  tensor(44)
When Input is  tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is  tensor(32)
'''
print()

In [98]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [99]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        #index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


i7J-8xDEtphSL:g_h,-mZfg05C_,VtU;Qv?*BaW*CF&1xn.Fbq_1z;za1KV]7Ywdi,z;;k_VhveIKPX:z(1?HO0Sd-*fP'f0zmc9uJMd4,mdpRTE&eFfxhhXEwc;c;oIz5?)A:;1M-﻿"k("[J)AH8rDUX:&:TT!oSCESZO*?P3y3mcxMg﻿blKgYo][g6"[mYpn3)!o8&rI7D]3DSk8eY!N[cG5ZONgB:EtoC1﻿W.91ehhyv!lMm4Dyv!Dzt5SkJB4"AuYs?m-0Lz*UStU":ZR25ek'FkjstROV﻿v49M_9Rm&XM1,ih8aZp*KM&.:lc2;dH nn3G,)]JE﻿4ZQiWhm_[tw4DRh
f]tnT1eFjsEb[f[UwQ
"_EoffWdpn0ST;G ?S7P3h!)A:DWuIG",z2JrO2[ulo
;IgY!F!!
p,zK--!FhLAJ
3VtGsWqmxDap q0
cz*f[?6dM1ewzSa﻿2JAYwxG HeafF,5fF5E-.rq,-!,SHlAlMj


In [125]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.530, val loss: 2.597
step: 250, train loss: 2.533, val loss: 2.579
step: 500, train loss: 2.521, val loss: 2.592
step: 750, train loss: 2.517, val loss: 2.570
2.179664134979248


In [126]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



ay. ntre isthaly a oa aned u d thexpo he Iblyotrentth po utins fthed Z6IMdeney.5 ath d tapiry th un tombe y. her "HZA"

S&N*[l pp thedivW9thyouted d wn'" he DWikes, tt."
s  hashered agUIthile,"Iy al!D!nd av[zx;Pve
fofe t?*[HQHY3n be r mp od t s?[ anle
aved thinuced, imullly wre hatolyS.

fthecur. the O(R?S
" wr b, Wind we s

", H
A2Dq
"mqF3!M-Ak id Jinoman f is;G0ley ad Bburerese d oce t ke per t agab. Cly he teed b, g t s D hib.
g hind hilfth "0t oloonacullfunoves,"I''5Zfe wreanobe therey  ted
